In [2]:
import pandas as pd
from connec_functions import GDB

### ICOS Carbon SPARQL Endpoint

#### Notes/Findings:
- sparql endpoint only queryable though browser interface, trying through code returns "501: Not Implemented"
- general queries take long time to run
- there is a set per-minute query quota
- RDF ~ data standardized
- Use of internally defined ontologies
  - requires knowledge on the location of the ontologies / vocabularies and getting to know them ~~ less machine interoperable
  - sidenote: prefixes not registered on prefix.cc ~ would increase findability (+ also registering in ontology catalogues/archives)
- Also use of known & common ontologies such as dcat (see example SPARQL query "DCAT metadata demo")  
*(note: to facilitate integration of data between different sources it would be better is data is described using common & well known ontologies/vocabularies + potential extension to those if domain specific information cannot be described by those ontologies)*

- use of known & trusted standard terms for mediaType and license (e.g. MIME-Types using iana.org, licenses using CC-urls)
- data granularity: file level
  - data files are accessible with dcat:downloadUrl within a dcat:distribution (data points themselves are not described in RDF)

In [3]:
# Define the SPARQL endpoint URL
sparql_endpoint_url = "https://meta.icos-cp.eu/sparql"
gdb = GDB(sparql_endpoint_url, "endpoint_queries")
gdb.execute_to_df("general.sparql") 
# curl https://meta.icos-cp.eu/sparql -X POST --data 'query=PREFIX%20cpmeta%3A%20%3Chttp%3A%2F%2Fmeta.icos-cp.eu%2Fontologies%2Fcpmeta%2F%3E%0APREFIX%20rdfs%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E%0APREFIX%20cpst%3A%20%3Chttp%3A%2F%2Fmeta.icos-cp.eu%2Fontologies%2Fstationentry%2F%3E%0ASELECT%0A%28coalesce%28%3FtcName%2C%20%3FhoName%29%20as%20%3FName%29%0A%3FTheme%0A%28coalesce%28%3FtcClass%2C%20%3FhoClass%29%20as%20%3FClass%29%0A%28coalesce%28%3FtcCountry%2C%20%3FhoCountry%29%20as%20%3FCountry%29%0AFROM%20%3Chttp%3A%2F%2Fmeta.icos-cp.eu%2Fresources%2Fstationentry%2F%3E%0AFROM%20%3Chttp%3A%2F%2Fmeta.icos-cp.eu%2Fontologies%2Fstationentry%2F%3E%0AFROM%20%3Chttp%3A%2F%2Fmeta.icos-cp.eu%2Fresources%2Ficos%2F%3E%0AWHERE%20%7B%0A%09%3Fs%20cpst%3AhasCountry%20%3FhoCountry%20%3B%20cpst%3AhasLongName%20%3FhoName%20%3B%20cpst%3AhasStationClass%20%3FhoClass0%20%3B%20a%2Frdfs%3Alabel%20%3FTheme%20.%0A%09optional%7B%0A%09%09%3Fs%20cpst%3AhasProductionCounterpart%20%3Fprods%20.%0A%09%09bind%28iri%28%3Fprods%29%20as%20%3Fprod%29%20.%0A%09%09optional%7B%3Fprod%20cpmeta%3AhasName%20%3FtcName%20%7D%0A%09%09optional%7B%3Fprod%20cpmeta%3AhasStationClass%20%3FtcClass%7D%0A%09%09optional%7B%3Fprod%20cpmeta%3AcountryCode%20%3FtcCountry%7D%0A%09%7D%0A%09bind%28if%28contains%28%3FhoClass0%2C%20%27Ass%27%29%2C%20%27Associated%27%2C%20%3FhoClass0%29%20as%20%3FhoClass%29%0A%09filter%20exists%20%7B%3Fs%20cpst%3AhasShortName%20%5B%5D%7D%0A%7D%0Aorder%20by%20%3FTheme%20%3FName'
# different compared to ARGO-ifremer SPARQL-endpoint (curl -X POST "https://sparql.ifremer.fr/argo/query" --data-urlencode "query=select ?s ?o ?p where{?s ?o ?p.} limit 10")
# curl -X POST "https://meta.icos-cp.eu/sparql" --data-urlencode "query=select ?s ?o ?p where{?s ?o ?p.} limit 10" --> does work ...

HTTPError: HTTP Error 501: Not Implemented

**Queries executed to obtain view on self-declared classes & properties in data model of ICOS data**

To obtain a view on the data model used to offer ICOS data as LD through the ICOS sparql endpoint, a number of sparql queries were executed and the results were then visualized as diagrams making use of Graphoo, which is a graphical framework for OWL ontologies ([specification](https://essepuntato.it/graffoo/)).

The first query returns classes and properties used in the stationentry ontology
```
SELECT DISTINCT ?s ?p ?o
FROM <http://meta.icos-cp.eu/ontologies/stationentry/>
WHERE {
	?s ?p ?o .
}
LIMIT 1000
```

![Local Image](images/ICOS_CP_datamodel.drawio.png)


The second query return classes and properties used in the cpmeta ontology
```
SELECT DISTINCT ?s ?p ?o 
FROM <http://meta.icos-cp.eu/ontologies/cpmeta/>
WHERE {
	?s ?p ?o .
}
LIMIT 2000
```

(this is a preliminary diagram of data model diagram (not all classes/predicates yet included))

![Local Image](images/ICOS_CP_datamodel2.drawio.png)


**A few instance examples:**

```
SELECT DISTINCT ?s ?p ?o
FROM <http://meta.icos-cp.eu/resources/stationentry/>
#FROM <http://meta.icos-cp.eu/ontologies/stationentry/>
#FROM <http://meta.icos-cp.eu/resources/icos/>
WHERE {
	<http://meta.icos-cp.eu/ontologies/stationentry/ES/IT-Noe> ?p ?o .
}
LIMIT 1000
```

![Local Image](images/ICOS_CP_datamodel-instance%20example.drawio.png)



```
SELECT DISTINCT ?p ?o
FROM <http://meta.icos-cp.eu/resources/stationentry/>
#FROM <http://meta.icos-cp.eu/ontologies/stationentry/>
#FROM <http://meta.icos-cp.eu/resources/icos/>
WHERE {
	<http://meta.icos-cp.eu/ontologies/stationentry/PI/spano@uniss.it> ?p ?o .
}
LIMIT 1000
```

![Local Image](images/ICOS_CP_datamodel-instance%20example%202.drawio.png)
